# The prediction model
Importing necessary libraries 

In [1]:
!pip install yake
!pip install stanza

import nltk, spacy, csv, re, yake, stanza, json
import pandas as pd
import numpy as np

from datetime import datetime
from keras_preprocessing.sequence import pad_sequences
from pprint import pprint
import tensorflow as tf

nltk.download('stopwords')
nltk.download('punkt')
stanza.download('en')

nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
kw_extractor = yake.KeywordExtractor()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rbnra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rbnra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2022-10-19 15:09:41 INFO: Downloading default packages for language: en (English) ...
2022-10-19 15:09:42 INFO: File exists: C:\Users\rbnra\stanza_resources\en\default.zip
2022-10-19 15:09:45 INFO: Finished downloading models and saved to C:\Users\rbnra\stanza_resources.
2022-10-19 15:09:45 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-10-19 15:09:46 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2022-10-19 15:09:46 INFO: Use device: cpu
2022-10-19 15:09:46 INFO: Loading: tokenize
2022-10-19 15:09:46 INFO: Loading: ner
2022-10-19 15:09:46 INFO: Done loading processors!


Dataset preprocessing

In [ ]:
with open('../Data/dic_allThreads.txt') as f:
    json_data = json.load(f)

data= []
for x in range(0, 500):
  first_key = list(json_data.values())[x]

  try:
    first_ = list(first_key.values())[1]
  except IndexError:
    first_ = list(first_key.values())[0]
  
  str_ = ''
  if first_[7]:
    for row in first_[7]:
      if '{' in row[0]:
        s = row[0].split('{', 1)[0]
        str_ = str_ + ' | ' + s
      else:
        str_ = str_ + ' | ' + row[0] 
    data.append(str_.replace(" ", "").replace("|", " ").strip().replace("_", ""))

print(data)

tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(data)
seq = tokenizer.texts_to_sequences(data)

X = []
y = []
total_words_dropped = 0

for i in seq:
    if len(i) > 1:
        for index in range(1, len(i)):
            X.append(i[:index])
            y.append(i[index])
    else:
        total_words_dropped += 1

Training the recommender model

In [3]:
X = tf.keras.preprocessing.sequence.pad_sequences(X)
y = tf.keras.utils.to_categorical(y)

vocab_size = len(tokenizer.word_index) + 1

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 14),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(vocab_size, activation='softmax'),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.02), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

model.fit(X, y, epochs=100)
model.save('nwp.h5')

Epoch 1/200
7/7 [==============================] - 2s 11ms/step - loss: 4.4675 - accuracy: 0.0577
Epoch 2/200
7/7 [==============================] - 0s 7ms/step - loss: 4.1339 - accuracy: 0.0721
Epoch 3/200
7/7 [==============================] - 0s 7ms/step - loss: 3.9415 - accuracy: 0.0721
Epoch 4/200
7/7 [==============================] - 0s 7ms/step - loss: 3.8924 - accuracy: 0.0913
Epoch 5/200
7/7 [==============================] - 0s 9ms/step - loss: 3.8667 - accuracy: 0.0913
Epoch 6/200
7/7 [==============================] - 0s 8ms/step - loss: 3.8726 - accuracy: 0.0673
Epoch 7/200
7/7 [==============================] - 0s 7ms/step - loss: 3.8459 - accuracy: 0.0962
Epoch 8/200
7/7 [==============================] - 0s 9ms/step - loss: 3.8450 - accuracy: 0.0913
Epoch 9/200
7/7 [==============================] - 0s 7ms/step - loss: 3.8460 - accuracy: 0.0913
Epoch 10/200
7/7 [==============================] - 0s 7ms/step - loss: 3.8401 - accuracy: 0.0913
Epoch 11/200
7/7 [==========

7/7 [==============================] - 0s 8ms/step - loss: 0.6460 - accuracy: 0.6971
Epoch 168/200
7/7 [==============================] - 0s 9ms/step - loss: 0.6539 - accuracy: 0.6875
Epoch 169/200
7/7 [==============================] - 0s 7ms/step - loss: 0.6510 - accuracy: 0.6635
Epoch 170/200
7/7 [==============================] - 0s 7ms/step - loss: 0.6485 - accuracy: 0.6827
Epoch 171/200
7/7 [==============================] - 0s 7ms/step - loss: 0.6458 - accuracy: 0.6923
Epoch 172/200
7/7 [==============================] - 0s 7ms/step - loss: 0.6387 - accuracy: 0.6971
Epoch 173/200
7/7 [==============================] - 0s 7ms/step - loss: 0.6569 - accuracy: 0.6731
Epoch 174/200
7/7 [==============================] - 0s 7ms/step - loss: 0.6460 - accuracy: 0.6971
Epoch 175/200
7/7 [==============================] - 0s 7ms/step - loss: 0.6476 - accuracy: 0.6779
Epoch 176/200
7/7 [==============================] - 0s 8ms/step - loss: 0.6592 - accuracy: 0.6779
Epoch 177/200
7/7 [=====

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 14)          1288      
                                                                 
 lstm (LSTM)                 (None, None, 100)         46000     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 92)                9292      
                                                                 
Total params: 147,080
Trainable params: 147,080
Non-trainable params: 0
_________________________________________________________________


In [5]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
generate_text("handle cover deal", 2, model, 10)